### N&auml;chster Punkt 
Gegeben sei ein Punkt `pt = (0, 6)` und eine Liste `pts` mit Punkten,  
`pts = [(0, 0), (0, 5), (0, 10)]`.  
Finde den Punkt in `pts`, der am n&auml;chsten bei `pt` liegt.  

1. Erstelle eine Liste mit Paaren der Form `(distance(pt, pts[i]), i)`:  
   `dist_idx = [(6, 0), (1, 1), (4, 2)]` 
1. Finde das kleinste Tuple `(1, 1)`   mit  `min(dist_idx)`  
  (Tuple werden in erster Priorit&auml;t nach dem ersten Element (hier die Distanz)  sortiert)
1. Greife den Index `1` des Punktes mit `(1, 1)[1]`  

Oft d&uuml;nnt man die Liste `dist_idx` aus und beh&auml;lt nur Punkte, welche im Umkreis einer Distanz `err` von `pt` liegen.


In [ ]:
def distance(p, q):
    return ((p[0] - q[0])**2 + (p[1] - q[1])**2)**0.5

def get_closest(pts, pt0, err):
    dist_idx = []
    for i, pt in enumerate(pts):
        dist = distance(pt, pt0)
        if dist < err:
            dist_idx.append((dist, i))
    
    return min(dist_idx)[1]

In [ ]:
pts = [(0, 0), (0, 5), (0, 10)]
pt = (0, 6)
idx = get_closest(pts, pt, err=2)
pts[idx]

In [ ]:
# Problem:
get_closest(pts, pt, err = 1)

### Verbesserung
Ist die Liste `dist_idx` leer weil der Punkt `pt` keinen Nachbarn im Umkreis `err` hat, 
verursacht `min(dist_idx)` einen Fehler.  
`min(dist_idx, defaul=(None, None))` liefert in diesem Fall das Tuple `(None, None)`.


In [1]:
#%%file get_closest.py
def distance(p, q):
    return ((p[0] - q[0])**2 + (p[1] - q[1])**2)**0.5

def get_closest(pts, pt0, err=10):
    dist_idx = []
    for i, pt in enumerate(pts):
        dist = distance(pt, pt0)
        if dist < err:
            dist_idx.append((dist, i))
    
    return min(dist_idx, default=(None, None))[1]

Writing get_closest.py


In [ ]:
print(get_closest(pts, pt, err = 1))

### Anwendung: Steine auf Leinwand setzen und verschieben

In [ ]:
pts = [(180, 100), (140, 30), (60, 30), (20, 100), (60, 170), (140, 170)]
       
def place_stone(canvas, idx, color='black'):
    x, y = pts[idx]
    canvas.fill_style = color
    canvas.fill_circle(x, y, radius)

def remove_stone(canvas, idx):
    x, y = pts[idx]
    canvas.clear_rect(x-radius-0.5, y-radius-0.5, 2*radius+1)

def move_stone(canvas, src, target, color='black'):
    remove_stone(canvas, src)
    place_stone(canvas, target, color)

In [ ]:
from ipywidgets import Output
err_msg = Output(layout = {'border': '1px solid black'})
from ipycanvas import MultiCanvas

colors = ['LightSalmon', 'Salmon', 'Red','Crimson',  'FireBrick', 'DarkRed']
radius = 10

mcanvas = MultiCanvas(2, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
bg, fg = mcanvas
fg.selected_field = None

@err_msg.capture()        
def on_mouse_down(x, y):
    idx = get_closest(pts, (x, y))
    if idx is not None:
        fg.selected_field = idx
        place_stone(fg, idx, colors[idx])

@err_msg.capture()   
def on_mouse_up(x, y):   
    if fg.selected_field is None:
        return
    target = get_closest(pts, (x, y))
    if target is not None:
        move_stone(fg, fg.selected_field, target, colors[target])  
        selected_field = None       
        
mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)

for pt in pts:
    bg.fill_style = 'blue'
    bg.fill_circle(*pt, 3)
    
display(mcanvas, err_msg)      